## Build Map Notebook
This notebook contains the functions from the lesson and provides a framework for extracting a map of the area based on the raw images:

* Load the recorded images from the Rover Simulator
For each frame apply the following:
* Perspective transformation of the image using the parameters extracted from the grid images
* Color Treshold of the image that filters out only the available areas
* Warp from camera-centric to rover-centric coordinates
* Rotate and Translate from rover-centric to the world coordinates

In [3]:
%matplotlib inline
#%matplotlib qt # Choose %matplotlib qt to plot to an interactive window (note it may show up behind your browser)
# Make some of the relevant imports
import cv2 # OpenCV for perspective transform
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import scipy.misc # For saving images as needed
import glob  # For reading in a list of images from a folder
import imageio
import pandas as pd
from IPython.display import HTML
from moviepy.editor import VideoFileClip
from moviepy.editor import ImageSequenceClip

image_width = 320
image_height = 160
scale = 10

**Color Treshold**

Function that takes as its input a color image and a 3-tuple of color threshold values (integer values from 0 to 255), and outputs a single-channel binary image, which is to say an image with a single color channel where every pixel is set to one or zero. In this case, all pixels that were above the threshold should be assigned a value of 1, and those below a value of 0

In [86]:
def color_thresh(img, rgb_thresh=(180, 180, 180)):
    # Create an array of zeros same xy size as img, but single channel
    color_select = np.zeros_like(img[:,:,0])
    # Require that each pixel be above all three threshold values in RGB
    # above_thresh will now contain a boolean array with "True"
    # where threshold was met
    above_thresh = (img[:,:,0] > rgb_thresh[0]) \
                & (img[:,:,1] > rgb_thresh[1]) \
                & (img[:,:,2] > rgb_thresh[2])
    # Index the array of zeros with the boolean array and set to 1
    color_select[above_thresh] = 1
    # Return the binary image
    return color_select

**Perspective Transformation**

From the image select the four points of a square in the robot camera view. Then using opencv deform the image to be on a flat surface and each square in the robot camera is mapped to a flat square in the deformed image

In [41]:
def perspect_transform(img):
    # Define calibration box in source (actual) and destination (desired) coordinates
    # These source and destination points are defined to warp the image
    # to a grid where each 10x10 pixel square represents 1 square meter
    dst_size = 5 
    # Set a bottom offset to account for the fact that the bottom of the image 
    # is not the position of the rover but a bit in front of it
    bottom_offset = 6
    src = np.float32([[14, 140], [301 ,140],[200, 96], [118, 96]])
    dst = np.float32([[image_width/2 - dst_size, image_height - bottom_offset],
                  [image_width/2 + dst_size, image_height - bottom_offset],
                  [image_width/2 + dst_size, image_height - 2*dst_size - bottom_offset], 
                  [image_width/2 - dst_size, image_height - 2*dst_size - bottom_offset],
                  ])

    # Get transform matrix using cv2.getPerspectivTransform()
    M = cv2.getPerspectiveTransform(src, dst)
    # Warp image using cv2.warpPerspective()
    # keep same size as input image
    warped = cv2.warpPerspective(img, M, (image_width, image_height))
    # Return the result
    return warped

**Rover Centric Coordinates**

Converts an image from Camera-Centric Coordinates to Rover-Centric Coordinates

In [42]:
def rover_coords(binary_img):
    # this function calculate pixel positions with reference to the rover 
    # position being at the center bottom of the image.  
    ypos, xpos = binary_img.nonzero()
    x_pixel = -(ypos - image_height).astype(np.float)
    y_pixel = -(xpos - image_width/2 ).astype(np.float)
    return x_pixel, y_pixel

**Rover Centric Coordinates to World**

Application of a rotation matrix composed by a rotation and a translation to the rover-centric map
in order to obtain the map in the world reference frame

In [43]:
# Define a function to apply a rotation to pixel positions
def rotate_pix(xpix, ypix, yaw):
    # Convert yaw to radians
    # Apply a rotation
    # yaw angle is recorded in degrees so first convert to radians
    yaw_rad = yaw * np.pi / 180
    xpix_rotated = xpix * np.cos(yaw_rad) - ypix * np.sin(yaw_rad)
    ypix_rotated = xpix * np.sin(yaw_rad) + ypix * np.cos(yaw_rad)
    # Return the result  
    return xpix_rotated, ypix_rotated

# Define a function to perform a translation
def translate_pix(xpix_rot, ypix_rot, xpos, ypos, scale): 
    # Apply a scaling and a translation
    # Assume a scale factor of 10 between world space pixels and rover space pixels
    # Perform translation and convert to integer since pixel values can't be float
    xpix_translated = np.int_(xpos + (xpix_rot / scale))
    ypix_translated = np.int_(ypos + (ypix_rot / scale))
    # Return the result  
    return xpix_translated, ypix_translated

# Define a function to apply rotation and translation (and clipping)
# Once you define the two functions above this function should work
def pix_to_world(xpix, ypix, xpos, ypos, yaw, world_size, scale):
    # Apply rotation
    xpix_rot, ypix_rot = rotate_pix(xpix, ypix, yaw)
    # Apply translation
    xpix_tran, ypix_tran = translate_pix(xpix_rot, ypix_rot, xpos, ypos, scale)
    # Clip to world_size
    x_pix_world = np.clip(np.int_(xpix_tran), 0, world_size - 1)
    y_pix_world = np.clip(np.int_(ypix_tran), 0, world_size - 1)
    # Return the result
    return x_pix_world, y_pix_world

## Read in saved data and ground truth map of the world
The next cell is all setup to read your saved data into a `pandas` dataframe.  Here you'll also read in a "ground truth" map of the world, where white pixels (pixel value = 1) represent navigable terrain.  

After that, we'll define a class to store telemetry data and pathnames to images.  When you instantiate this class (`data = Databucket()`) you'll have a global variable called `data` that you can refer to for telemetry and map data within the `process_image()` function in the following cell. 

In [44]:
# Read in csv file as a dataframe

df = pd.read_csv('../Recordings/robot_log.csv', delimiter=';', decimal='.')
csv_img_list = df["Path"].tolist() # Create list of image pathnames
# Read in ground truth map and create a 3-channel image with it
ground_truth = mpimg.imread('../Recordings/calibration_images/map_bw.png')
ground_truth_3d = np.dstack((ground_truth*0, ground_truth*255, ground_truth*0)).astype(np.float)

# Creating a class to be the data container
# Will read in saved data from csv file and populate this object
# Worldmap is instantiated as 200 x 200 grids corresponding 
# to a 200m x 200m space (same size as the ground truth map: 200 x 200 pixels)
# This encompasses the full range of output position values in x and y from the sim
class Databucket():
    def __init__(self):
        self.images = csv_img_list  
        self.xpos = df["X_Position"].values
        self.ypos = df["Y_Position"].values
        self.yaw = df["Yaw"].values
        self.count = 0 # This will be a running index
        self.worldmap = np.zeros((200, 200, 3)).astype(np.float)
        self.ground_truth = ground_truth_3d # Ground truth worldmap

# Instantiate a Databucket().. this will be a global variable/object
# that you can refer to in the process_image() function below
data = Databucket()

## Function to processes stored images

The `process_image()` function below adds perception step processes (functions defined above) to perform image analysis and mapping.  The following cell is all set up to use this `process_image()` function in conjunction with the `moviepy` video processing package to create a video from the images you saved taking data in the simulator.  

The function `process_image()` will be passed individual images into and outputs an image called `output_image` that will be stored as one frame of video.  You can make a mosaic of the various steps of your analysis process and add text as you like (example provided below).  

In [87]:
# Define a function to pass stored images to
# reading rover position and yaw angle from csv file
# This function will be used by moviepy to create an output video
def process_image(img):
    # Use the Databucket() object defined above
    # to print the current x, y and yaw values 
    rover_xpos = data.xpos[data.count]
    rover_ypos = data.ypos[data.count]
    rover_yaw = data.yaw[data.count]

    # 1) Apply perspective transform
    warped = perspect_transform(img)
    
    # 2) Apply color threshold to identify navigable terrain/obstacles/rock samples
    colored = color_thresh(warped)
    
    # 3) Convert thresholded image pixel values to rover-centric coords
    xpix, ypix = rover_coords(colored)
    
    # 4) Convert rover-centric pixel values to world coords
    x_world, y_world = pix_to_world(xpix, ypix, rover_xpos, 
                                rover_ypos, rover_yaw, 
                                data.worldmap.shape[0], scale)
    
    # 5) Update worldmap (to be displayed on right side of screen)
        # Example: data.worldmap[obstacle_y_world, obstacle_x_world, 0] += 1
        #          data.worldmap[rock_y_world, rock_x_world, 1] += 1
        #          data.worldmap[navigable_y_world, navigable_x_world, 2] += 1
    data.worldmap[y_world, x_world, 0] = 255

    # 6) Make a mosaic image, below is some example code
        # First create a blank image (can be whatever shape you like)
    output_image = np.zeros((img.shape[0] + data.worldmap.shape[0], img.shape[1]*2, 3))
        # Next you can populate regions of the image with various output
        # Here I'm putting the original image in the upper left hand corner
    output_image[0:img.shape[0], 0:img.shape[1]] = img
    
        # Add the warped image in the upper right hand corner
    output_image[0:img.shape[0], img.shape[1]:] = warped
    
        # Add the constructed map in the lower right hand corner
    output_image[img.shape[0]:, img.shape[1]:img.shape[1]+data.worldmap.shape[1]] = data.worldmap

        # Overlay worldmap with ground truth map
    map_add = cv2.addWeighted(data.worldmap, 1, data.ground_truth, 0.5, 0)
        # Flip map overlay so y-axis points upward and add to output_image 
    output_image[img.shape[0]:, 0:data.worldmap.shape[1]] = np.flipud(map_add)


        # Then putting some text over the image
    cv2.putText(output_image,"Populate this image with your analyses to make a video!", (20, 20), 
                cv2.FONT_HERSHEY_COMPLEX, 0.4, (255, 255, 255), 1)
    if data.count < len(data.images) - 1:
        data.count += 1 # Keep track of the index in the Databucket()
    
    return output_image

## Make and Play a video from processed image data
Use the [moviepy](https://zulko.github.io/moviepy/) library to process images and create a video.

After that, use an inline video player to play the video

In [88]:
# Import everything needed to edit/save/watch video clips
# Define pathname to save the output video
output = 'output/test_mapping.mp4'
data = Databucket() # Re-initialize data in case you're running this cell multiple times
clip = ImageSequenceClip(data.images, fps=60) # Note: output video will be sped up because 
                                          # recording rate in simulator is fps=25
new_clip = clip.fl_image(process_image) #NOTE: this function expects color images!!
%time new_clip.write_videofile(output, audio=False)

t:   0%|          | 2/1561 [00:00<01:41, 15.37it/s, now=None]

Moviepy - Building video output/test_mapping.mp4.
Moviepy - Writing video output/test_mapping.mp4



Moviepy - Done !
Moviepy - video ready output/test_mapping.mp4
Wall time: 14.8 s


In [89]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output))